In [2]:
from planet4 import io, region_data, markings
from p4_tools import get_final_markings_counts
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
from scipy.optimize import root, curve_fit, leastsq
import pdb
import shapely.geometry as shp
import fiona as fio
from shapely import affinity
from shapely.ops import cascaded_union, unary_union
import time

# read out metadata for season 2 and 3
meta = pd.read_csv('ithaca_metadata.csv')

# read out actual markings for fans and blotches, both seasons combined
fans = pd.read_csv('ithaca_fans.csv')
blotches = pd.read_csv('ithaca_blotches.csv')

# read out tile coordinate data
tiles = pd.read_csv('ithaca_tile_coords.csv')

# find unique obsids in the fans and blotches catalogs
fimg_names = fans.image_name.unique()
bimg_names = blotches.image_name.unique()
fimg_id = fans.image_id.unique()
bimg_id = blotches.image_id.unique()

# add column for tile info, fill with tile info
fans['tile_lat'] = 0
fans['tile_lon'] = 0
blotches['tile_lat'] = 0
blotches['tile_lon'] = 0
for i in range(len(fimg_id)):
    fans.tile_lat[fans.image_id==fimg_id[i]] = float(tiles.PlanetocentricLatitude[tiles.image_id==fimg_id[i]])
    fans.tile_lon[fans.image_id==fimg_id[i]] = float(tiles.PositiveEast360Longitude[tiles.image_id==fimg_id[i]])
for i in range(len(bimg_id)):
    blotches.tile_lat[blotches.image_id==bimg_id[i]] = float(tiles.PlanetocentricLatitude[tiles.image_id==bimg_id[i]])
    blotches.tile_lon[blotches.image_id==bimg_id[i]] = float(tiles.PositiveEast360Longitude[tiles.image_id==bimg_id[i]])
    
# add column for season flag and validity of marking
fans['season'] = 0
fans['valid_marking'] = True
blotches['season'] = 0
blotches['valid_marking'] = True
blotches['area'] = np.pi * blotches.radius_1 * blotches.radius_2 / 4

# define column for season flag
fans.season[fans.obsid.str[5] == '1'] = 2
fans.season[fans.obsid.str[5] == '2'] = 3
blotches.season[blotches.obsid.str[5] == '1'] = 2
blotches.season[blotches.obsid.str[5] == '2'] = 3

# find what size fans should be removed for fair comparison
min_fan_pixels = fans.distance.min()
min_bl_area = blotches.area.min()

max_scale = np.max( meta.map_scale.max() )
print('maximum map_scale:',  max_scale, '; min_fan_marking:', min_fan_pixels, '; min_bl_area:', min_bl_area)

# if the minimal fan marking tool is = 10 pixels at max_bin = 4, it will correspond to  
# min_fan_pixels * max_binning / image_binning
meta['min_fan'] = min_fan_pixels * meta.map_scale.max() // meta.map_scale + 1

# if the minimal blotch marking tool is = 80 sq. pixels at max_bin = 4, it will correspond to  
# min_bl_area * max_binning^2 / image_binning^2
meta['min_bl'] = min_bl_area * meta.map_scale.max()**2 // meta.map_scale**2 + 1

# mark "valid_marking" key to be False for blotch markings smaller than min_bl for that image
print(len(blotches))
for i in range(len(blotches)):
    nr_image =  np.where(meta.obsid == blotches.obsid[i])[0][0]        
    min_b = meta.min_bl[ nr_image  ]         
    if (blotches.area[i] < min_b ):
        blotches.valid_marking[i] = False
            
# mark "valid_marking" key to be False for fan markings smaller than min_fan for that image
for i in range(len(fans)):
    nr_image =  np.where(meta.obsid == fans.obsid[i])[0][0]        
    min_f = meta.min_fan[ nr_image  ]         
    if (fans.distance[i] < min_f ):
        fans.valid_marking[i] = False

/Users/chha0593/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/chha0593/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:38: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/chha0593/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/chha0593/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:41: SettingWithCopyWarning: 
A value is trying

maximum map_scale: 1.0 ; min_fan_marking: 10.0 ; min_bl_area: 78.5398163397
18045


/Users/chha0593/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [3]:
# create combined 'objects' dataframe with fans and blotches, mark the type of each object, and re-sort by name
objects = fans[fans.valid_marking==True].merge(blotches[blotches.valid_marking==True],how='outer')
objects.loc[:len(fans.valid_marking[fans.valid_marking==True]),'type'] = 0
objects.loc[len(fans.valid_marking[fans.valid_marking==True]):,'type'] = 1
objects = objects.sort_values(by='image_name').reset_index(drop=True)
img_unique = objects.image_name.unique()
objects.head(10)

,angle,distance,image_id,image_name,image_x,image_y,n_votes,spread,version,vote_ratio,...,y_tile,obsid,tile_lat,tile_lon,season,valid_marking,radius_1,radius_2,area,type
0,208.810277,89.036506,APF0000an2,ESP_011350_0945,107.085714,34055.285714,63,78.662848,1.0,NaN,...,63.0,ESP_011350_0945,-85.064139,181.157947,2,True,NaN,NaN,NaN,0.0
1,227.334474,34.734543,APF0000aqj,ESP_011350_0945,936.000000,29137.800000,5,92.275315,1.0,0.51,...,54.0,ESP_011350_0945,-85.109760,181.166044,2,True,NaN,NaN,NaN,0.0
2,73.084765,110.208772,APF0000aqj,ESP_011350_0945,785.750000,29292.250000,4,64.493874,1.0,NaN,...,54.0,ESP_011350_0945,-85.109760,181.166044,2,True,NaN,NaN,NaN,0.0
3,214.571707,79.848686,APF0000aqj,ESP_011350_0945,1457.500000,29381.500000,4,79.295934,1.0,NaN,...,54.0,ESP_011350_0945,-85.109760,181.166044,2,True,NaN,NaN,NaN,0.0
4,212.871005,69.128720,APF0000aqj,ESP_011350_0945,1250.428571,29565.142857,7,99.408254,1.0,NaN,...,54.0,ESP_011350_0945,-85.109760,181.166044,2,True,NaN,NaN,NaN,0.0
5,216.385412,113.992901,APF0000aqj,ESP_011350_0945,1378.000000,29529.875000,8,86.912324,1.0,NaN,...,54.0,ESP_011350_0945,-85.109760,181.166044,2,True,NaN,NaN,NaN,0.0
6,237.295218,71.628725,APF0000aqj,ESP_011350_0945,1026.472222,29333.222222,18,84.184539,1.0,NaN,...,54.0,ESP_011350_0945,-85.109760,181.166044,2,True,NaN,NaN,NaN,0.0
7,225.786517,57.053910,APF0000aqj,ESP_011350_0945,1172.045455,29233.909091,11,79.403162,1.0,NaN,...,54.0,ESP_011350_0945,-85.109760,181.166044,2,True,NaN,NaN,NaN,0.0
8,235.190207,55.877282,APF0000aqj,ESP_011350_0945,829.375000,29657.000000,4,76.170208,1.0,NaN,...,54.0,ESP_011350_0945,-85.109760,181.166044,2,True,NaN,NaN,NaN,0.0
9,204.612260,33.820821,APF0000aqj,ESP_011350_0945,1167.375000,29591.250000,4,114.163124,1.0,NaN,...,54.0,ESP_011350_0945,-85.109760,181.166044,2,True,NaN,NaN,NaN,0.0


In [4]:
# create collection of shapely polygons representing 'objects'
collection = [None] * len(objects)
points = [None] * 14
for k in range(len(objects)):
    if objects.type[k]==0:
        r = objects.distance[k]*np.tan((objects.spread[k]/2.)*np.pi/180.)
        red_ax = objects.distance[k] - r
        points[0] = (objects.image_x[k],objects.image_y[k])
        points[1] = (objects.image_x[k] + r,objects.image_y[k] + red_ax)
        points[2] = (objects.image_x[k] + r*np.cos(15.*np.pi/180.),objects.image_y[k] + r*np.sin(15.*np.pi/180.) + red_ax)
        points[3] = (objects.image_x[k] + r*np.cos(30.*np.pi/180.),objects.image_y[k] + r*np.sin(30.*np.pi/180.) + red_ax)
        points[4] = (objects.image_x[k] + r*np.cos(45.*np.pi/180.),objects.image_y[k] + r*np.sin(45.*np.pi/180.) + red_ax)
        points[5] = (objects.image_x[k] + r*np.cos(60.*np.pi/180.),objects.image_y[k] + r*np.sin(60.*np.pi/180.) + red_ax)
        points[6] = (objects.image_x[k] + r*np.cos(75.*np.pi/180.),objects.image_y[k] + r*np.sin(75.*np.pi/180.) + red_ax)
        points[7] = (objects.image_x[k] + r*np.cos(90.*np.pi/180.),objects.image_y[k] + r*np.sin(90.*np.pi/180.) + red_ax)
        points[8] = (objects.image_x[k] + r*np.cos(105.*np.pi/180.),objects.image_y[k] + r*np.sin(105.*np.pi/180.) + red_ax)
        points[9] = (objects.image_x[k] + r*np.cos(120.*np.pi/180.),objects.image_y[k] + r*np.sin(120.*np.pi/180.) + red_ax)
        points[10] = (objects.image_x[k] + r*np.cos(135.*np.pi/180.),objects.image_y[k] + r*np.sin(135.*np.pi/180.) + red_ax)
        points[11] = (objects.image_x[k] + r*np.cos(150.*np.pi/180.),objects.image_y[k] + r*np.sin(150.*np.pi/180.) + red_ax)
        points[12] = (objects.image_x[k] + r*np.cos(165.*np.pi/180.),objects.image_y[k] + r*np.sin(165.*np.pi/180.) + red_ax)
        points[13] = (objects.image_x[k] + r*np.cos(180.*np.pi/180.),objects.image_y[k] + r*np.sin(180.*np.pi/180.) + red_ax)
        collection[k] = shp.Polygon(points[:])
        collection[k] = affinity.rotate(collection[k],angle=objects.angle[k] - 90.,origin=points[0])
    else:
        circle = shp.point.Point(objects.image_x[k],objects.image_y[k]).buffer(1)
        collection[k] = affinity.scale(circle,objects.radius_1[k],objects.radius_2[k])
        collection[k] = affinity.rotate(collection[k],angle=objects.angle[k])

I was not sure if the lookup table should also keep track of the type of each object so I included some code that can be commented out that marks a 0 (for fans) and a 1 (for blotches) as the first entry in every dictionary index

In [5]:
# d is the lookup table (dictionary)
d = {}
# loop thru images
for n in range(len(img_unique)-13):
    rng = len(np.where(objects.image_name == img_unique[n])[0])
    if n == 0:
        # start is to keep track of where to start indexing in 'collection' as we step thru images
        start = rng
        # loop thru objects in image, checking against each other object in image
        for i in range(rng):
            d[i] = []
            d[i].append(objects.type[i])
            for j in range(rng-i-1):
                # if the center of object [i] is within another object [j], the index [j] is recorded at index [i] in 'd'
                if collection[i].centroid.intersects(collection[j+i+1]):
                    d[i].append(j+i+1)
    else:
        for i in range(rng):
            d[start+i] = []
            d[start+i].append(objects.type[start+i])
            for j in range(rng-i-1):
                if collection[start+i].centroid.intersects(collection[start+j+i+1]):
                    d[start+i].append(start+j+i+1)
        start = rng + start
        
print(d)

{0: [0.0, 1442], 1: [0.0], 2: [0.0, 22, 49, 572, 789, 1710], 3: [0.0, 26], 4: [0.0, 25, 587], 5: [0.0, 19, 582], 6: [0.0], 7: [0.0, 597], 8: [0.0, 15, 108, 808, 1705], 9: [0.0, 587, 1134], 10: [0.0], 11: [0.0, 588], 12: [0.0], 13: [0.0, 588, 589], 14: [0.0, 30, 588], 15: [0.0, 108, 588, 808], 16: [0.0], 17: [0.0, 1223], 18: [0.0, 581], 19: [0.0, 582], 20: [0.0, 1134], 21: [0.0], 22: [0.0, 572], 23: [0.0], 24: [0.0, 588], 25: [0.0], 26: [0.0], 27: [0.0, 584], 28: [1.0, 1135, 1139], 29: [1.0, 598, 1121], 30: [1.0, 588], 31: [1.0, 658], 32: [1.0, 33, 1099], 33: [1.0, 1099], 34: [1.0, 1107, 1176, 1178], 35: [1.0, 1180], 36: [1.0], 37: [1.0, 657], 38: [1.0, 835, 862, 1103, 1109], 39: [1.0], 40: [1.0], 41: [1.0], 42: [1.0, 44, 1196], 43: [1.0, 744, 1025], 44: [1.0, 1196], 45: [1.0, 1173], 46: [1.0, 182, 1175, 1199, 1433, 1440], 47: [1.0, 1151, 1152], 48: [1.0], 49: [0.0, 572, 789, 1710], 50: [0.0, 575], 51: [0.0, 790], 52: [0.0], 53: [0.0], 54: [0.0], 55: [0.0, 60], 56: [0.0], 57: [0.0, 512,